In [1]:
from osgeo import gdal
from osgeo import ogr
import pandas as pd
import geopandas as gpd
from sklearn.model_selection import train_test_split
import rasterio
from sklearn.model_selection import KFold
import os

In [21]:
df = gpd.read_file("data/snowpoint.shp")
df.head()

,STATION,SNOWDEPTH,geometry
0,111,17,POINT (746057.688 4319837.500)
1,112,19,POINT (747707.688 4320227.500)
2,113,17,POINT (747677.688 4319207.500)
3,114,19,POINT (738467.688 4314557.500)
4,115,12,POINT (738137.688 4325327.500)


In [22]:
df = df.set_geometry('geometry')

In [23]:
n=1
kf = KFold(n_splits=5, random_state=128, shuffle=True)

for train_index, test_index in kf.split(df):
    X_train, X_test = df.iloc[train_index], df.iloc[test_index]
    X_train.to_file('TrainTest/trainSnowPoints'+str(n)+'.shp')
    X_test.to_file('TrainTest/testSnowPoints'+str(n)+'.shp')
 
    nn = gdal.Grid("Results/Nearest"+str(n)+".tif", 'TrainTest/trainSnowPoints'+str(n)+'.shp', zfield="SNOWDEPTH",
                algorithm = "nearest")
    nn = None
    ma = gdal.Grid("Results/Average"+str(n)+".tif", 'TrainTest/trainSnowPoints'+str(n)+'.shp', zfield="SNOWDEPTH",
                algorithm = "average")
    ma = None
    idw = gdal.Grid("Results/IDW"+str(n)+".tif", 'TrainTest/trainSnowPoints'+str(n)+'.shp', zfield = "SNOWDEPTH",
                    algorithm = "invdist:power=6")
    idw = None

    n+=1

print(n)


6


## RMSE Score

In [8]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [25]:
def Normalization(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(data)
    newNorm=scaler.transform(data)
    return newNorm


In [26]:
files=["IDW","Average","Nearest"]

for file in files:
    summation=0
    for i in range(1,6):
        pts = gpd.read_file('TrainTest/testSnowPoints'+str(i)+'.shp')
        pts = pts[['STATION','SNOWDEPTH','geometry']]
        pts.index = range(len(pts))
        coords = [(x,y) for x, y in zip(pts.geometry.x, pts.geometry.y)]

        src = rasterio.open("Results/"+str(file)+str(i)+".tif")

        pts['PredictedDepth'] = [x[0] for x in src.sample(coords)]
        actual=pts["SNOWDEPTH"].array.reshape(-1,1)
        predicted=pts['PredictedDepth'].array.reshape(-1,1)
        actual=Normalization(actual)
        predicted=Normalization(predicted)
        summation+=mean_squared_error(actual, predicted, squared=False)
    average=summation/5
    print("The average of",file, "is", average)

The average of IDW is 0.21278774571545994
The average of Average is 0.6027491423176513
The average of Nearest is 0.2562385575318728


In [27]:
src = gdal.Open("data/arelev1.tif")
ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
lrx = ulx + (src.RasterXSize * xres)
lry = uly + (src.RasterYSize * yres)

lin = gdal.Grid("tifResults/SnowDepth.tif", "data/snowpoint.shp", zfield = "SNOWDEPTH", algorithm = "invdist:power=6",width=752,height=948)
lin = None

## Calulcating Slope and HillShade

In [28]:
def calculate_slope(DEM):
    gdal.DEMProcessing('tifResults/slope.tif', DEM, 'slope')
    with rasterio.open('tifResults/slope.tif') as dataset:
        slope=dataset.read(1)
    return slope
def calculate_hillshade(DEM):
    gdal.DEMProcessing('tifResults/Hillshade.tif', DEM, 'hillshade')
    with rasterio.open('tifResults/Hillshade.tif') as dataset:
        hillshade=dataset.read(1)
    return hillshade
def calculate_aspect(DEM):
    gdal.DEMProcessing('tifResults/aspect.tif', DEM, 'aspect')
    with rasterio.open('tifResults/aspect.tif') as dataset:
        aspect=dataset.read(1)
    return aspect

slope=calculate_slope("data/arelev1.tif")
hillshade=calculate_hillshade("data/arelev1.tif")
aspect=calculate_aspect("data/arelev1.tif")
slop=None
hillshade=None



In [11]:
def Normy(data):
    scaler = MinMaxScaler(feature_range=(0, 10))
    scaler.fit(data)
    newNorm=scaler.transform(data)
    return newNorm


In [12]:
result_path = 'tifResults/'
X_RES, Y_RES = 948,752

In [13]:
rasters = dict()
for name in os.listdir(result_path):
   rasters[name.split('.')[0]] = gdal.Open(result_path+name)

In [14]:
rasters

{'aspect': <osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000025D3FB80CF0> >,
 'Hillshade': <osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000025D3F85A400> >,
 'slope': <osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000025D3F85A190> >,
 'SnowDepth': <osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000025D3F859770> >}

In [15]:
final_raster = Normy(rasters['SnowDepth'].GetRasterBand(1).ReadAsArray())

In [16]:
for criteria in rasters.keys():
    if criteria != 'SnowDepth':    
        final_raster += Normy(rasters[criteria].GetRasterBand(1).ReadAsArray())

In [17]:
final_ds = gdal.GetDriverByName('GTiff').Create("final_raster.tiff", Y_RES, X_RES, 1, gdal.GDT_Byte)


In [18]:
final_ds.SetGeoTransform(rasters['aspect'].GetGeoTransform())
final_ds.SetProjection(rasters['aspect'].GetProjection())

0

In [19]:
final_band = final_ds.GetRasterBand(1)
final_band.SetNoDataValue(-9999)
final_band.WriteArray(final_raster, 0, 0)
final_band.FlushCache()

In [38]:
for file in rasters.values():
    del file

del final_ds
del final_band